In [1]:
# import block
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance

from coding_1 import *

In [2]:
# import data
records = pd.read_csv("Ecommerce.csv")
print("shape of records is", records.shape)
records.head()

shape of records is (541909, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


In [3]:
records_small = records[["CustomerID", "Quantity", "UnitPrice", "Country"]].dropna()
records_small["CustomerID"] = records_small["CustomerID"].astype(int)

# calculate total pay for each invoice
records_small["Total"] = records_small["Quantity"] * records_small["UnitPrice"]
records_small

# convert the country column to numerical
records_small["Country"] = pd.Categorical(records_small["Country"])
records_small["Country"] = records_small["Country"].cat.codes

print("shape of records_small is", records_small.shape)
records_small.head(15)

shape of records_small is (406829, 5)


,CustomerID,Quantity,UnitPrice,Country,Total
0,17850,6,2.55,35,15.30
1,17850,6,3.39,35,20.34
2,17850,8,2.75,35,22.00
3,17850,6,3.39,35,20.34
4,17850,6,3.39,35,20.34
5,17850,2,7.65,35,15.30
6,17850,6,4.25,35,25.50
7,17850,6,1.85,35,11.10
8,17850,6,1.85,35,11.10
9,13047,32,1.69,35,54.08


In [4]:
# calculate the total amount of money each customer spent
# code consultation:
# https://stackoverflow.com/questions/49783178/python-keep-other-columns-when-using-sum-with-groupby

customer = records_small.groupby(["CustomerID"], as_index=False)['Quantity', 'Total', 'Country'].agg({'Quantity': 'sum', 'Total': 'sum', 'Country': 'first'})
print("shape of customer is", customer.shape)
customer.head()

shape of customer is (4372, 4)
<ipython-input-4-e3973b21bd30>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  customer = records_small.groupby(["CustomerID"], as_index=False)['Quantity', 'Total', 'Country'].agg({'Quantity': 'sum', 'Total': 'sum', 'Country': 'first'})


,CustomerID,Quantity,Total,Country
0,12346,0,0.00,35
1,12347,2458,4310.00,16
2,12348,2341,1797.24,12
3,12349,631,1757.55,18
4,12350,197,334.40,24


In [5]:
# convert the dataset to a numpy array for later use
customer_np = customer[['Quantity', 'Total', 'Country']].to_numpy()
customer_np

array([[   0.  ,    0.  ,   35.  ],
       [2458.  , 4310.  ,   16.  ],
       [2341.  , 1797.24,   12.  ],
       ...,
       [  98.  ,  176.6 ,   35.  ],
       [1397.  , 2094.88,   35.  ],
       [1586.  , 1837.28,   35.  ]])

In [9]:
# test my_kmeans(data, k, random)

my_km = my_kmeans(customer_np, 5, 2021)
print(len(my_km[1]))
my_km[0]

4372


array([[2.36090000e+04, 3.83386427e+04, 3.48461538e+01],
       [1.09956667e+05, 2.41136560e+05, 3.10000000e+01],
       [6.42753333e+04, 9.77254617e+04, 2.08333333e+01],
       [5.47153789e+02, 9.13085924e+02, 3.32983851e+01],
       [4.40781410e+03, 7.43053785e+03, 3.19423077e+01]])

In [11]:
# looping k-means with implementation from sklearn

SSE_list = looping_kmeans(customer_np, range(1, 16))
SSE_list

[407.76746479582243,
 95.481796389734,
 78.9658609821125,
 62.062531685897795,
 59.0999674983127,
 53.11022990979635,
 51.61726374452886,
 47.00683312751935,
 43.86783258903348,
 43.03677510183669,
 28.6332765489159,
 35.62018422501967,
 31.633607856425733,
 21.513108276288804,
 20.596434344385443]